### Segment Geospatial (samgeo)

In [ ]:
#conda install pytorch torchvision -c pytorch
#pip install segment-geospatial groundingdino-py leafmap localtileserver
#conda install gdal


In [ ]:
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM
# import leafmap


In [ ]:
# imagem local

image = "D:\\drone_img_sam\\levantamento 2309-1\\jpg\\0800.jpeg"

In [ ]:
# dividir em pedaços menores, lidar com restrições de memória

split_raster(image, out_dir="tiles", tile_size=(2000, 2000), overlap=10)

In [ ]:
# carregar o modelo de segmentação

sam = LangSAM(model_type='vit_h')

In [ ]:
text_prompt = "person"

box_t = 0.35

text_t =0.35

In [ ]:
mascaras = sam.predict_batch(
    images='tiles',
    out_dir='masks',
    text_prompt=text_prompt,
    box_threshold=box_t,
    text_threshold=text_t,
    mask_multiplier=255,
    dtype='uint8',
    merge=True,
    verbose=True,
)

In [ ]:
# criar um mapa para visualizar os resultados

# m = leafmap.Map(center=[-27.4349, -48.3917], zoom=17, height="500px")
# m.add_basemap("SATELLITE")
# m.add_raster(image, nodata=0, layer_name='Img')
# m.add_layer_manager()

In [ ]:
# adicionar a máscara final a visualização

# m.add_raster('masks/merged.tif', cmap='summer', nodata=0, layer_name='Mask')
# m

In [ ]:
# exportar a máscara para um shapefile

sam.raster_to_vector("masks/merged.tif", "output/merged.shp")

In [ ]:
# style = {
#     "color": "#3388ff",
#     "weight": 2,
#     "fillColor": "#7c4185",
#     "fillOpacity": 0.5,
# }
# m.add_vector("output/merged.shp", layer_name="Vector", style=style)
# m